# Experiments on creation of global counterfactual explanations using nearest neighbors algorithm.
Our goal is to generate global counterfactual explanations to describe the changes that need to happen to the features values in order to change the classification outcome.

To accomplish this we take the next steps:
- We start by identifying two datasets that we want to compare, and use the k-NN algorithm to find the nearest neighbors in the second dataset for each instance of the first dataset.
- We create a third dataset that contains only the unique neighbors from the second dataset that were found in step 1.
- We run a k-NN algorithm on the third dataset to find the nearest neighbor for each instance in the third dataset.
- If all instances in the third dataset have a distance of 0 to their nearest neighbor, it suggests that all instances in the third dataset are identical in terms of their features, and we may be able to generate one global counterfactual that represents the feature changes necessary for an instance from the first dataset to be classified with the target label of the second dataset.

### Dataset used : Adult Dataset<br>
The target label is to predict whether an individual earns more or less than $50,000 per year based on the other features in the dataset. 




In [1]:
from scipy.spatial import cKDTree
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import xgboost
import sklearn
from aif360.sklearn.datasets import fetch_adult,fetch_bank,fetch_compas,fetch_german
from cflib import *

### Adult Dataset

In [2]:
X,y = load_dataset('adult')
X.shape

(45222, 13)

In [3]:
duplicates=X.duplicated()
duplicates.value_counts()

False    38229
True      6993
dtype: int64

In [4]:
#annotate duplicate rows
X['duplicated'] = X.duplicated()
df = create_df(X,y,'adult')

In [5]:
#drop duplicate rows
indexdupl = df[(df['duplicated'] == True)].index
df.drop(indexdupl , inplace=True)
df = df.drop(columns='duplicated')
df.shape

(38229, 13)

### Most occured value for each feature in the dataset

In [6]:
most_occured(df)

,Data,Most occurred Category,Percentage (%)
0,age,33.0,2.83
1,workclass,Private,70.24
2,education-num,9.0,30.31
3,marital-status,Married-civ-spouse,44.55
4,occupation,Prof-specialty,14.07
5,relationship,Husband,38.63
6,race,White,83.90
7,sex,Male,65.84
8,capital-gain,0.0,90.19
9,capital-loss,0.0,94.45


### Preprocessing
- Create bins for numerical features such as age and hours per week
- Convert categorical data to numerical with one hot enconding

In [7]:
df = preprocess(df,'adult') 
df.head()

,age,workclass,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
0,25.0,2,7.0,4,6,3,2,1,0.0,0.0,40.0,38,0
1,38.0,2,9.0,2,4,0,4,1,0.0,0.0,50.0,38,0
2,28.0,1,12.0,2,10,0,4,1,0.0,0.0,40.0,38,1
3,44.0,2,10.0,2,6,0,2,1,7688.0,0.0,40.0,38,1
4,34.0,2,6.0,4,7,1,4,1,0.0,0.0,30.0,38,0


### Create dataframes for privileged and unprivileged groups based on which target label they belong and which protected attribute we select.
For instance, if we select 'sex' as protected attribute the privileged group is 'Male' and the unprivileged group is 'Female'.<br>

Affected instances are the ones that have a value of target label equal to 0 ('<=50K' per year)<br>
Unaffected instances are the ones that have a value of target label equal to 1 ('>50K' per year)<br>

- Affected Privileged - affected_A
- Unaffected Privileged - not_affected A

- Affected Unprivileged - affected_B
- Unaffected Unprivileged - not_affected_B

In [8]:
#Loukas: Protected attribute na ginei parametros, protected na metonomastei se sensitive
#Loukas: Ekana metonomasia stis metavlites apo edw kai katw
#grouping instances based on label and prottected attribute
#For example, dataframe of only male(privileged) instances with label 0(affected) and dataframe of only male(privileged) instances with label 1(unaffected)
sensitive_attribute = 'sex'
affected_A, affected_B, not_affected_A, not_affected_B = group(df,sensitive_attribute=sensitive_attribute) 

In [9]:
affected_A.head()

,age,workclass,education-num,marital-status,occupation,relationship,race,capital-gain,capital-loss,hours-per-week,native-country
0,25.0,2,7.0,4,6,3,2,0.0,0.0,40.0,38
1,38.0,2,9.0,2,4,0,4,0.0,0.0,50.0,38
2,34.0,2,6.0,4,7,1,4,0.0,0.0,30.0,38
3,55.0,2,4.0,2,2,0,4,0.0,0.0,10.0,38
4,36.0,0,13.0,2,0,0,4,0.0,0.0,40.0,38


### First step : For the affected instances find their nearest neighbor in the dataset of unaffected.
In order to do that we organize the unaffected groups in KDTrees. <br>
A KDTree is a data structure for organizing points in a k-dimensional space. For each instance of the affected we then query inside the KDTree in order to find its nearest neighbor in the unaffected.


In [10]:
#creation of kdtrees
tree_A = KDTree(not_affected_A)
tree_B = KDTree(not_affected_B)



### For each instance of affected datasets, query in the KDTrees we created in order to find their nearest neighbor.

We created `nearest` function to find the nearest neighbors.
To do that the function uses `kdtree.query` which takes 3 variables:
- x : an array of points to query, in our problem variable x is the instances of the affected.
- k : the number of nearest neighbors to return, we choose k=1.
- p : which distance to use in order to compute the nearest neighbors. 1 is the sum-of-absolute-values distance and 2 is the usual Euclidean distance.

Our function returns the following variables :
- match : contains the distances between each instance and its neighbor
- result : contains the instance of affected, instance of unaffected and their distance
- index : contains the index of the nearest neighboor in the dataset of unaffected

In [11]:
distances_A, result_A, neighbor_index_A = nearest(affected_A,tree_A,not_affected_A)
distances_B, result_B, neighbor_index_B = nearest(affected_B,tree_B,not_affected_B)

# print(result_A[0])
# print(neighbor_index_A[0])
# print(affected_A.iloc[0])
# print(affected_A.iloc[1])
# unpriv_match, result_unpriv, index_unpriv = nearest(aff_unprivileged,un_unpriv_tree,unaff_unprivileged)

In [12]:
#dataframes that contain for each instance of the affected, the index of their nearest neighbor.
indexes_A = pd.DataFrame(neighbor_index_A , columns=['Neighbor']) 
indexes_B = pd.DataFrame(neighbor_index_B , columns=['Neighbor'])

#concat distances in index dataframe
indexes_A['Distance'] = distances_A
indexes_B['Distance'] = distances_B

indexes_A

# crazy=indexes_A.loc[indexes_A['Distance']==0]
# print(crazy)
# print(affected_A.iloc[7], not_affected_A.iloc[4593])
#find number of instances that have a neighbor at 0 distance
#Loukas: Auto einai terma koulo, Na mazepsoume 5 tetoia kai na ta typwseis sto original format tou dataframe
# print(f"There are { indexes_A['distances'].value_counts()[indexes_A['distances'].value_counts().index==0.0].values[0]} of privileged affected instances that have a neighbor at distance 0")
# print(f"There are { indexes_B['distances'].value_counts()[indexes_B['distances'].value_counts().index==0.0].values[0] } of unprivileged affected instances that have a neighbor at distance 0")

,Neighbor,Distance
0,2100,3.741657
1,1666,1.000000
2,1222,6.324555
3,772,8.062258
4,6117,1.414214
...,...,...
17290,3654,1.414214
17291,7051,4.000000
17292,11,1.414214
17293,5389,8.185353


There is a strong possibility that more than one instance of the affected has the same instance of unaffected as their neighbor.<br>
In the below cells we explore this and if that's the case we select the instances that correspond only to the unique neighbors. 

In [13]:
#some instances of the affected may have the same unaffected instance as neighbor
#below we create the lists that contain the indexes of the unique neighbors
unique_A = indexes_A['Neighbor'].value_counts().index.to_list() 
unique_B = indexes_B['Neighbor'].value_counts().index.to_list()

# indexes_A['Neighbor'].count()

# print(f"There are { len(unique_A) } unique neighbors that correspond to {len(affected_A)} affected privileged instances")
# print(f"There are { len(unique_B) } unique neighbors that correspond to {len(affected_B)} affected unprivileged instances")

### Second step: After finding the unique neighbors, we store them in dataframes.

In [14]:
#For privileged group
unique_A = not_affected_A.loc[unique_A]
unique_A.head()

,age,workclass,education-num,marital-status,occupation,relationship,race,capital-gain,capital-loss,hours-per-week,native-country
589,22.0,2,10.0,4,11,1,4,0.0,0.0,25.0,38
7185,30.0,2,14.0,2,7,0,4,0.0,0.0,15.0,38
1101,20.0,2,8.0,4,7,3,2,0.0,0.0,35.0,38
2100,24.0,2,7.0,4,9,3,4,0.0,0.0,40.0,38
6456,22.0,2,10.0,4,7,3,4,0.0,0.0,32.0,38


In [15]:
#For unprivileged group
unique_B = not_affected_B.loc[unique_B]
unique_B.head()

,age,workclass,education-num,marital-status,occupation,relationship,race,capital-gain,capital-loss,hours-per-week,native-country
444,24.0,4,13.0,2,9,2,4,0.0,0.0,20.0,38
1286,23.0,2,9.0,2,6,5,4,0.0,0.0,40.0,38
730,25.0,2,13.0,2,3,5,4,0.0,0.0,30.0,38
993,24.0,2,14.0,2,9,5,4,0.0,0.0,35.0,38
715,28.0,2,12.0,2,3,5,4,0.0,0.0,16.0,38


### Third Step: Use KNN algorithm to each of the above datasets,containing the unique neighbors of the affected, and find for each instance their closest neighbor. 
Our goal in this step is to generate the global counterfactuals.<br>
The above datasets contain the unique neighbors of the affected groups. By finding in each dataset the distances between their instances we can generate the global counterfactuals.<br>
For example, if we find only 5 instances with neighbors at distance > 0 and all the other instances have a neighbor at distance 0, we can generate 6 global counterfactuals.

To explore this solution we created `nn_closest()` function that uses `sklearn.neighbors.NearestNeighbors` to implement a neighbor search inside the dataset
 <br>Our function returns for each instance:
 - the index of its neighbor
 - the distance between them

In [16]:
closest_A = nn_closest(unique_A)
closest_A 
# we added to the dataframe a column that contains the index of the closest neighbor of the instance and a column with the distance between them

,age,workclass,education-num,marital-status,occupation,relationship,race,capital-gain,capital-loss,hours-per-week,native-country,indexes,distances
589,22.0,2,10.0,4,11,1,4,0.0,0.0,25.0,38,64,4.358899
7185,30.0,2,14.0,2,7,0,4,0.0,0.0,15.0,38,95,5.567764
1101,20.0,2,8.0,4,7,3,2,0.0,0.0,35.0,38,4,4.582576
2100,24.0,2,7.0,4,9,3,4,0.0,0.0,40.0,38,8,4.690416
6456,22.0,2,10.0,4,7,3,4,0.0,0.0,32.0,38,2,4.582576
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1690,43.0,2,13.0,2,3,0,4,0.0,0.0,47.0,38,3449,2.236068
1034,29.0,2,9.0,2,2,0,4,0.0,0.0,45.0,38,253,1.000000
5903,41.0,2,14.0,2,9,0,4,0.0,0.0,55.0,38,3494,2.236068
395,58.0,2,13.0,6,2,1,4,0.0,0.0,40.0,38,2812,2.645751


In [17]:
print(f"The unique neighbors of the affected privileged group have { closest_A['distances'].value_counts()[closest_A['distances'].value_counts().index<=10000].values[0]} close instances.")

The unique neighbors of the affected privileged group have 833 close instances.


In [18]:
closest_B = nn_closest(unique_B)
closest_B.head()
# we added to the dataframe a column that contains the index of the closest neighbor of the instance and a column with their distance

,age,workclass,education-num,marital-status,occupation,relationship,race,capital-gain,capital-loss,hours-per-week,native-country,indexes,distances
444,24.0,4,13.0,2,9,2,4,0.0,0.0,20.0,38,196,5.916080
1286,23.0,2,9.0,2,6,5,4,0.0,0.0,40.0,38,12,2.000000
730,25.0,2,13.0,2,3,5,4,0.0,0.0,30.0,38,1002,3.741657
993,24.0,2,14.0,2,9,5,4,0.0,0.0,35.0,38,392,3.464102
715,28.0,2,12.0,2,3,5,4,0.0,0.0,16.0,38,10,5.477226


In [19]:
print(f"The unique neighbors of the affected unprivileged group have { closest_B['distances'].value_counts()[closest_B['distances'].value_counts().index<=100000].values[0]} close instances")

The unique neighbors of the affected unprivileged group have 89 close instances
